In [1]:
import sys
import imp
sys.path.insert(0, '../src')

In [2]:
import denoise.graph.io as graphio
import denoise.graph.operations as graphops
imp.reload(graphio)

edgelist, node_list, node_map = graphio.parse_graph_file("../data/dream_files/1_ppi_string_v2.txt")
G = graphops.densify(edgelist)

print(G[node_map["PRKAG2"], node_map["ZNF559-ZNF177"]]) # Should output 0.064
print(G[node_map["ARL5C"], node_map["YKT6"]])           # Should output 0.15

_, proteins_to_go = graphio.parse_go_label_file("../data/dream_files/human.golabels")

print(proteins_to_go["PRKAG2"]) # Works as expected
print(proteins_to_go["YKT6"])   # Works as expected

0.064
0.15
['GO:0016772', 'GO:0016740', 'GO:0016301', 'GO:0140096', 'GO:0004672', 'GO:0016773', 'GO:0004674', 'GO:0060089']
['GO:0031090', 'GO:0098588', 'GO:0000139', 'GO:0044431']


In [15]:
import numpy as np
from denoise import predict
imp.reload(predict)

W = np.zeros((6, 6))
M = 1_000_000

# Node names and labels
A, B, C, D, E, F = range(0, 6)
labels = {
    A: ["P3"],
    B: ["P1", "P2"],
    E: ["P3", "P4"],
    F: ["P4", "P2"]
}
labels_f = lambda n: (labels[n] if n in labels else []) 

W[A, B] = M
W[B, C] = 4
W[A, C] = 1
W[D, A] = 3
W[D, C] = M
W[E, D] = 1

W = W + W.T # make undirected

# Expected Output: {0: 'P3', 1: 'P1', 2: 'P1', 3: 'P3', 4: 'P3', 5: 'P4'}
predict.wmv(W, labels_f)

NODE: 2 VOTER: 0
NODE: 2 VOTER: 1
NODE: 2 VOTER: 3
NODE: 3 VOTER: 0
NODE: 3 VOTER: 2
NODE: 3 VOTER: 4


{0: 'P3', 1: 'P1', 2: 'P1', 3: 'P3', 4: 'P3', 5: 'P4'}

In [16]:
from denoise import scoring
from denoise import predict
imp.reload(predict)
imp.reload(scoring)

n, _ = G.shape
num_labeled = 0
for i in range(n):
    if node_list[i] not in proteins_to_go:
        num_labeled += 1
print(f"Unlabelled over labelled: {num_labeled}/{n}")

labels = {i: proteins_to_go[node_list[i]] for i in range(n)
          if node_list[i] in proteins_to_go}

def predictor(training_labels):
    tlabels_f = lambda i: (training_labels[i] if i in training_labels else [])
    return predict.wmv(G, tlabels_f)

scoring.kfoldcv(2, labels, predictor, randomized=True)

AttributeError: 'int' object has no attribute 'shape'